# Demo arquitecturas avanzadas

## Modelo master-slave

### Dependencias

In [1]:
import socket
import threading

### Master

In [ ]:
def master(task):
    # Crear un socket TCP/IP
    master_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    # Conectarse al Slave
    master_socket.connect(('localhost', 9999))

    try:
        # Enviar la tarea al Slave
        master_socket.sendall(task.encode())
        print(f"Master - Envió tarea: {task}")

        # Esperar la respuesta del Slave
        result = master_socket.recv(1024).decode()
        print(f"Master - Recibió resultado: {result}")
    finally:
        # Cerrar la conexión
        master_socket.close()


### Slave

In [ ]:
def slave():
    # Crear un socket TCP/IP
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('localhost', 9999))
    server_socket.listen(1)
    print("Slave esperando tareas...")

    while True:
        # Esperar a que un cliente se conecte
        connection, client_address = server_socket.accept()
        try:
            print(f"Slave - Conexión establecida con {client_address}")

            # Recibir la tarea (mensaje) del Master
            task = connection.recv(1024).decode()
            print(f"Slave - Recibió tarea: {task}")

            # Procesar la tarea (simulación)
            result = f"Resultado de {task}"

            # Enviar el resultado de vuelta al Master
            connection.sendall(result.encode())
        finally:
            # Cerrar la conexión
            connection.close()

In [ ]:
slave_thread = threading.Thread(target=slave, daemon=True)
slave_thread.start()

Slave esperando tareas...


In [ ]:
# Enviar una tarea desde el Master al Slave
master("calcular la suma de 2+2")


Slave - Conexión establecida con ('127.0.0.1', 48384)
Slave - Recibió tarea: calcular la suma de 2+2
Master - Envió tarea: calcular la suma de 2+2
Master - Recibió resultado: Resultado de calcular la suma de 2+2


## Entrenamiendo AI/ML en CPUs, GPUs y TPUs

### Dependencias

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np


### Dataset

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


### Acceso a TPUs para entrenamiento

In [3]:
# Intenta encontrar y configurar una TPU
try:
    # Resolver la TPU disponible en el entorno, especialmente en Google Colab.
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()

    # Conectar al clúster de TPU, lo que permite que TensorFlow coordine las comunicaciones entre los nodos de la TPU.
    tf.config.experimental_connect_to_cluster(resolver)

    # Inicializar el sistema TPU para que esté listo para usar. Esto configura la TPU para el entrenamiento.
    tf.tpu.experimental.initialize_tpu_system(resolver)

    # Imprime todos los dispositivos lógicos que corresponden a la TPU.
    # Esto es útil para verificar si las TPUs se han detectado correctamente.
    print("All devices: ", tf.config.list_logical_devices('TPU'))

    # Define una estrategia de distribución para usar la TPU, lo que permite el uso de TPUs para la distribución de tareas.
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

# Si no se encuentra ninguna TPU, se captura la excepción ValueError
except ValueError:
    # Si no se encuentra una TPU, se usa la estrategia por defecto, que puede ser CPU o GPU según lo que esté disponible.
    print("No TPU found")
    strategy = tf.distribute.get_strategy()

No TPU found


### Definición del modelo (CNN) y Entrenamiento

In [4]:
with strategy.scope():

    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy']) # Now 'accuracy' is created within the TPUStrategy scope

    model.fit(train_images,
              train_labels,
              epochs=10,
              validation_data=(test_images, test_labels))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 5ms/step - accuracy: 0.3549 - loss: 1.7477 - val_accuracy: 0.5504 - val_loss: 1.2541
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.5766 - loss: 1.1841 - val_accuracy: 0.6119 - val_loss: 1.0929
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.6443 - loss: 1.0103 - val_accuracy: 0.6057 - val_loss: 1.1255
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.6821 - loss: 0.9128 - val_accuracy: 0.6691 - val_loss: 0.9386
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7058 - loss: 0.8384 - val_accuracy: 0.6744 - val_loss: 0.9267
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7284 - loss: 0.7719 - val_accuracy: 0.7000 - val_loss: 0.8572
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7515 - loss: 0.7126 - val_accuracy: 0.7000 - val_loss: 0.8783
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7644 - loss: 0.6749 

### Evaluación

In [5]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc}')


313/313 - 0s - 2ms/step - accuracy: 0.7181 - loss: 0.8451

Test accuracy: 0.7181000113487244
